In [1]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt 
from collections import defaultdict
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler, WeightedRandomSampler

In [2]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

is_cuda = torch.cuda.is_available()

# If we have a GPU available, set our device to GPU. use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")   

Found GPU at: /device:GPU:0
GPU is available


In [3]:
# import torchtext.vocab

# glove = torchtext.vocab.GloVe(name='6B', dim=100)

# print(f'There are {len(glove.itos)} words in the vocabulary')

# glove.vectors.shape

#glove.vectors[glove.stoi['natural']]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
# data = pd.read_csv('/content/drive/My Drive/train_snli.txt', sep="\t", header=None)
# data.columns = ["ans1", "ans2", "class"]
data = pd.read_csv('/content/drive/My Drive/Clean_train_data.csv')
# Display 10 random rows from the data.
data.sample(10)

,Questions,ans1,ans2,class
2221,Explain the layer of CNN: ReLU,"Once a feature map is created, we can pass eac...",LSTM use persistent previous information to be...,0
4044,What is a friend function?,operator that takes three operands: a conditio...,A friend function of a class is defined outsid...,0
3200,"What are PCA, KPCA, and ICA used for?",PCA (Principal Components Analysis) is like KP...,they are commonly used for dimensionality redu...,1
2389,What is the softmax function?,RNN) are the state of the art algorithm for se...,Softmax turns arbitrary real values into proba...,0
1547,What do you understand by Tensors?,A tensor is a container which can house data i...,Tensors are mathematical objects that generali...,1
1268,What is the softmax function?,The Softmax regression is a form of logistic r...,"The input values can be positive, negative, ze...",1
162,What is Model Capacity?,supervised learning is where I have the input ...,The capacity of a deep learning neural network...,0
631,Why is zero initialization not a good weight i...,If the set of weights in the network is put to...,"in this case, the equations of the learning al...",1
1507,What do you understand by Tensors?,Tensors are nothing but a de facto for represe...,The Rectified Linear Unit is the most commonly...,0
4193,What is Software configuration management?,"It is a technic of identifying, organizing, an...",Software requirements are a functional descrip...,0


In [34]:
def clean_text(text):
    #Make text lowercase, remove text in square brackets,remove links,remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    return text

def clean2_text(text):
  #remove some of stopwords as 'a, an, the', ... 
  txt = clean_text(text)
  words = txt.split(' ')
  aft_remove = [w for w in words if w not in ['a', 'an', 'the', 'of', 'that', 'which','in', 'at', 'for', 'where', 'when', 'to', 'from']]
  return ' '.join(words)


# cl_tx = clean2_text(p)
# print(cl_tx)

In [35]:
#Convert train data into list of tuples where each tuple is of the form (sentence1, sentence2)
sentences_pair = []
labels = []
for _, row in data.iterrows():
    
    s1 = clean2_text(str(row['ans1']))
    s2 = clean2_text(str(row['ans2']))
    label = int(row['class'])
    
    if s1 and s2:
        sentences_pair.append(( s1, s2 ))
        labels.append(label)

print ('Train Data sentences Pairs: ', len(sentences_pair))
print(sentences_pair[0])

Train Data sentences Pairs:  4274
('deep learning is a part of machine learning with an algorithm inspired by the structure and function of the brain which is called an artificial neural network deep learning is suited over a range of fields such as computer vision speech recognition natural language processing etc ', 'deep learning is a subset of machine learning in artificial intelligence that has networks capable of learning unsupervised from data that is unstructured or unlabeled also known as deep neural learning or deep neural network ')


In [36]:
Vocab = {}
n_words = 0

#build vocab using training data
for data in [sentences_pair]:
  for sentence_pair in data:
      s1 = sentence_pair[0]
      s2 = sentence_pair[1]
      for word in s1.split(' '):
        if word not in Vocab:
          Vocab[word] = n_words + 1
          n_words += 1
      
      for word in s2.split(' '):
        if word not in Vocab:
          Vocab[word] = n_words + 1
          n_words += 1
    
    
print("Total words in vocab are",len(Vocab))
# print(Vocab)  

Total words in vocab are 2560


In [37]:
class SentencesDataset(Dataset):
    def __init__(self, sentences_list, word2index, labels):
        self.sentences_list = sentences_list
        self.labels = labels
        self.word2index = word2index
        
    def __len__(self):
        return len(self.sentences_list)
    
    def __getitem__(self, index):
        sentences_pair = self.sentences_list[index]
        s1 = sentences_pair[0]
        s1_indices = []
        for word in s1.split():
            s1_indices.append(self.word2index[word])
            
        s2 = sentences_pair[1]
        s2_indices = []
        for word in s2.split():
            s2_indices.append(self.word2index[word])
            
        # s1_indices and s2_indices are lists of indices against words used in the sentence 
        return s1_indices, s2_indices, self.labels[index]
    
train_dataset = SentencesDataset(sentences_pair, Vocab, labels)
# print(len(train_dataset))

In [38]:
class CustomCollate:
    def custom_collate(self, batch):

        # batch = list of tuples where each tuple is of the form ([i1, i2, i3], [j1, j2, j3], label)
        s1_list = []
        s2_list = []
        labels = []
        for training_example in batch:
          s1_list.append(training_example[0])
          s2_list.append(training_example[1])
          labels.append(training_example[2])
          
        
        s1_lengths = [len(s) for s in s1_list]
        s2_lengths = [len(s) for s in s2_list]
        
        return s1_list, s1_lengths, s2_list, s2_lengths, labels

    def __call__(self, batch):
        return self.custom_collate(batch)

In [39]:
validation_split = 0.1
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
shuffle_dataset = True
random_seed = 32

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(val_indices)

batch_size = 32

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, collate_fn=CustomCollate())
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=validation_sampler, collate_fn=CustomCollate())

print ('Training Set Size {}, Validation Set Size {}'.format(len(train_indices), len(val_indices)))

Training Set Size 3847, Validation Set Size 427


In [19]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [20]:
vec_king = wv['king']
wv.vectors

array([[ 1.1291504e-03, -8.9645386e-04,  3.1852722e-04, ...,
        -1.5640259e-03, -1.2302399e-04, -8.6307526e-05],
       [ 7.0312500e-02,  8.6914062e-02,  8.7890625e-02, ...,
        -4.7607422e-02,  1.4465332e-02, -6.2500000e-02],
       [-1.1779785e-02, -4.7363281e-02,  4.4677734e-02, ...,
         7.1289062e-02, -3.4912109e-02,  2.4169922e-02],
       ...,
       [-1.9653320e-02, -9.0820312e-02, -1.9409180e-02, ...,
        -1.6357422e-02, -1.3427734e-02,  4.6630859e-02],
       [ 3.2714844e-02, -3.2226562e-02,  3.6132812e-02, ...,
        -8.8500977e-03,  2.6977539e-02,  1.9042969e-02],
       [ 4.5166016e-02, -4.5166016e-02, -3.9367676e-03, ...,
         7.9589844e-02,  7.2265625e-02,  1.3000488e-02]], dtype=float32)

In [40]:
# Convert word2vec embeddings into FloatTensor
word2vec_model = wv
word2vec_weights = torch.FloatTensor(word2vec_model.vectors)
embeddingSize = 300

weights = torch.randn(n_vocabulary_words+1, embeddingSize)
weights[0] = torch.zeros(embeddingSize)
for word, lang_word_index in Vocab.items():
    if word in word2vec_model:
        weights[lang_word_index] = torch.FloatTensor(word2vec_model.word_vec(word))

# del word2vec_model
del word2vec_weights

In [41]:
class SiameseNetwork(nn.Module):
    def __init__(self, pretrained_weights, hidden_dim, embeddingSize, no_layers):
        super(SiameseNetwork, self).__init__()

        self.hidden_dim = hidden_dim
        self.embeddingSize = embeddingSize
        self.no_layers = no_layers

        # Embedding layers from the pre-trained weights
        self.embedding = nn.Embedding.from_pretrained(pretrained_weights)
        self.embedding.weight.requires_grad = False
        # Create a single LSTM since this is a Siamese Network and the weights are shared
        self.lstm = nn.LSTM(input_size=embeddingSize, hidden_size=hidden_dim, num_layers = no_layers, batch_first = True)
    
    # Manhattan Distance Calculator
    def exponent_neg_manhattan_distance(self, x1, x2):
        return torch.exp(-torch.sum(torch.abs(x1 - x2), dim=0)).to(device)

    def forward_once(self, x, input_lengths, hidden):
      
        # Reverse sequence lengths indices in decreasing order before Padding and Packing
        sorted_indices = np.flipud(np.argsort(input_lengths))
        input_lengths = np.flipud(np.sort(input_lengths))
        input_lengths = input_lengths.copy() 
        
        # Reorder questions in the decreasing order of their lengths
        ordered_sentences = [torch.LongTensor(x[i]).to(device) for i in sorted_indices]
        # Pad sequences with 0s to the max length sequence in the batch
        ordered_sentences = torch.nn.utils.rnn.pad_sequence(ordered_sentences, batch_first=True)

      
        embeddings = self.embedding(ordered_sentences).to(device)
        
        # Pack the padded sequences and pass it through LSTM
        packed = torch.nn.utils.rnn.pack_padded_sequence(embeddings, input_lengths, batch_first=True)
        out, hidden = self.lstm(packed, hidden)
        unpacked, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True, total_length=int(input_lengths[0]))
        
        # The following step reorders the calculated activations to the original order in which sentences were passed
        result = torch.FloatTensor(unpacked.size())
        for i, encoded_matrix in enumerate(unpacked):
            result[sorted_indices[i]] = encoded_matrix
        return result

    def forward(self, s1, s1_lengths, s2, s2_lengths, hidden):
        output1 = self.forward_once(s1, s1_lengths, hidden)  #shape of output1 (batch_size, sequence length, hidden_dim)
        output2 = self.forward_once(s2, s2_lengths, hidden)  #shape of output2 (batch_size, sequence length, hidden_dim)

        # Calculate Similarity Score between both sentences in a single pair
        similarity_score = torch.zeros(output1.size()[0]).to(device)
        for index in range(output1.size()[0]):
            s1 = output1[index, s1_lengths[index] - 1, :]
            s2 = output2[index, s2_lengths[index] - 1, :]
            similarity_score[index] = self.exponent_neg_manhattan_distance(s1, s2)
        return similarity_score

    def init_hidden(self, batch_size):
        # initialize hidden states with sizes n_layers x batch_size x hidden_dim
        # initialized to zero, for hidden state and cell state of LSTM
        h0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        c0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden    


In [42]:
no_layers = 1
embeddingSize = 300
hidden_dim = 128

model = SiameseNetwork(weights, hidden_dim, embeddingSize, no_layers)
#moving to gpu
model.to(device)
print(model)

SiameseNetwork(
  (embedding): Embedding(2752, 300)
  (lstm): LSTM(300, 128, batch_first=True)
)


In [43]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01 )

# function to predict accuracy
def acc(pred,label):
    pred = torch.round(pred.squeeze())
    return torch.sum(pred == label.squeeze()).item()

In [44]:
# Threshold 0.5. Since similarity score will be a value between 0 and 1, we will consider all sentences pair with values greater than threshold as the same meaning
num_epochs = 5
clip = 5

valid_loss_min = np.Inf
# train for some number of epochs
epoch_tr_loss,epoch_vl_loss = [],[]
epoch_tr_acc,epoch_vl_acc = [],[]

for epoch in range(num_epochs):
    train_losses = []
    train_acc = 0.0
    model.train()

    # initialize hidden state
    hidden = model.init_hidden(batch_size)

    for batch_i, (s1_batch, s1_batch_lengths, s2_batch, s2_batch_lengths, labels) in enumerate(train_loader, 1):

        n_batches = len(train_indices)//batch_size
        if(batch_i > n_batches):
            break

        labels = torch.FloatTensor(labels).to(device)
        h = tuple([each.data for each in hidden])
        
        # Clear grads
        optimizer.zero_grad()
        
        # Run the forward pass
        similarity_score = model(s1_batch, s1_batch_lengths, s2_batch, s2_batch_lengths, hidden)

        # Calculate Loss
        loss = criterion(similarity_score, labels)
        # Calculate gradients
        loss.backward()
                
        # clip_grad_norm prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        # calculating accuracy
        accuracy = acc(similarity_score,labels)

        train_losses.append(loss.item())
        train_acc += accuracy
        
            
    val_h = model.init_hidden(batch_size)
    val_losses = []
    val_acc = 0.0
    model.eval()
    # with torch.no_grad():
    for batch_i, (s1_batch, s1_batch_lengths, s2_batch, s2_batch_lengths, labels) in enumerate(val_loader, 1):
        # make sure iterate over completely full batches, only
        n_batches = len(val_indices)//batch_size
        if(batch_i > n_batches):
            break

        labels = torch.FloatTensor(labels).to(device)

        similarity_score = model(s1_batch, s1_batch_lengths, s2_batch, s2_batch_lengths, val_h)
        
        val_loss = criterion(similarity_score, labels)
        val_losses.append(val_loss.item())

        accuracy = acc(similarity_score,labels)
        val_acc += accuracy
        
           
    epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(val_losses)  

    epoch_train_acc = train_acc/len(train_indices)
    epoch_val_acc = val_acc/len(val_indices)
    epoch_tr_loss.append(epoch_train_loss)
    epoch_vl_loss.append(epoch_val_loss)
    epoch_tr_acc.append(epoch_train_acc)
    epoch_vl_acc.append(epoch_val_acc)
    print(f'Epoch {epoch}') 
    print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')
    print(f'train_accuracy : {epoch_train_acc*100} val_accuracy : {epoch_val_acc*100}')
    if epoch_val_loss <= valid_loss_min:
        # torch.save(model.state_dict(), path)
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
        valid_loss_min = epoch_val_loss
    print(25*'==')

Epoch 0
train_loss : 0.2265402643630902 val_loss : 0.21481963189748618
train_accuracy : 69.30075383415648 val_accuracy : 67.68149882903981
Validation loss decreased (inf --> 0.214820).  Saving model ...
Epoch 1
train_loss : 0.17901735939085484 val_loss : 0.19744295111069313
train_accuracy : 73.61580452300494 val_accuracy : 68.38407494145198
Validation loss decreased (0.214820 --> 0.197443).  Saving model ...
Epoch 2
train_loss : 0.15130097679793836 val_loss : 0.17829346542175
train_accuracy : 79.02261502469456 val_accuracy : 71.19437939110071
Validation loss decreased (0.197443 --> 0.178293).  Saving model ...
Epoch 3
train_loss : 0.13099400339027245 val_loss : 0.16336578417282838
train_accuracy : 83.18170002599427 val_accuracy : 74.00468384074942
Validation loss decreased (0.178293 --> 0.163366).  Saving model ...
Epoch 4
train_loss : 0.11378925588602821 val_loss : 0.15246669833476728
train_accuracy : 87.15882505848714 val_accuracy : 78.22014051522248
Validation loss decreased (0.1633